Tips from http://blog.qure.ai/notes/semantic-segmentation-deep-learning-review
- Use encoder-decoder, upsampling by doing a reverse convolution and concatenating with the feature maps that were before the downsampling or by copying the index of the max-pooling
- Use VGG convolution layers trained on imagenet and remove the final FC layers or use ResNet for the encoder.
- Use dillated convolution rather than downsampling (they still do some pooling for downsampling, but most of the increase in field of view is from the dillatation)
- It is common to have a predicted segmentation map that is 1/8th of the input image size. One uses interpolation to get the final segmentation map.
- Use a fully connected CRF after interpolation (trained separately)
- Use residual conv unit, multi-resolution fusion and chained residual pooling to combine multi-resolution input
- Very large kernel are useful for the classification part of the network. They play a role similar as the FC layer at the end of a classifier. They are better to use a large receptive field that deep network like ResNet even though ResNet has theoretically a very large receptive field. kxk convolution are approximated by 1xk+kx1 convolutions to reduce the number of parameter and the computational expense.

What I think about this particular case (road detection):
- It is not needed to have large field of view, but it is important to have a good resolution.

In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from keras import layers, regularizers
#from keras import models
from keras.models import Model, Sequential
from keras import callbacks
from time import time
from keras import backend as K
#import tensorflow as tf
K.set_image_data_format("channels_first")

Using TensorFlow backend.


In [2]:
modelList = []
history = []
xaxis = []
class TimeHistory(callbacks.Callback):
    def __init__(self, seconds=np.inf):
        super(callbacks.Callback, self).__init__()
        self.seconds = seconds
    def on_train_begin(self, logs={}):
        self.times = []
        self.start = time()
    def on_epoch_end(self, batch, logs={}):
        self.times.append(time() - self.start)
        if time() - self.start > self.seconds: # if not defined, there is no limit since it is np.inf
            self.model.stop_training = True
            print('Stopping after %s seconds.' % self.seconds)
                
def Plot(i, text = 'acc', label = ''):
    plt.plot(xaxis[i], history[i].history['val_'+text], label = label)
    
def retrain(i_model, batchsize=99, nepochs=10, timelimit=np.inf):
    time_callback = TimeHistory(timelimit)
    es = EarlyStopping(monitor='val_loss', min_delta=0., patience=10)
    history[i_model].history = list(history[i_model].history)
    history[i_model].history.append(modelList[i_model].fit_generator(generator(batchsize), 
                    steps_per_epoch(989-yVal.shape[0])//batchsize,
                   epochs=nepochs, verbose=2, validation_data=([xMulVal,xPanVal],yVal),callbacks=[time_callback, es]))
    xaxis[i_model].append(time_callback.times)
    
def generator(batchsize):
    while True:
        for i in range(10):
            print('  file', i)
            xMul = np.load(r'..\data\xMul'+str(i)+'.npy')
            xPan = np.load(r'..\data\xPan'+str(i)+'.npy')
            y = np.load(r'..\data\y325_'+str(i)+'.npy') # I will use the 325x325 resolution for the target
            xMul = (xMul - 414.1505843473475)/271.51272549425477
            xPan = (xPan - 532.8910415421709)/318.8023222567299
            j=0
            while j<xMul.shape[0]:
                print('    ',j, '/', xMul.shape[0])
                yield ([xMul[j:j+batchsize], xPan[j:j+batchsize]], y[j:j+batchsize])
                j+= batchsize
                
# load Validation data
xMulVal = np.load(r'..\data\xMulVal.npy')
xPanVal = np.load(r'..\data\xPanVal.npy')
yVal = np.load(r'..\data\yVal325.npy')
xMulVal = (xMulVal - 414.1505843473475)/271.51272549425477
xPanVal = (xPanVal - 532.8910415421709)/318.8023222567299

print(xMulVal.shape,xPanVal.shape, yVal.shape)

def train(batchsize=99, nepochs=10, timelimit=np.inf):
    XMul = layers.Input(shape=xMulVal.shape[1:])
    XPan = layers.Input(shape=xPanVal.shape[1:])
    
    # Actual neural netowrk given by function NN
    Y = NN(XMul,XPan)
    
    model = Model(inputs=[XMul,XPan], outputs=Y)

    time_callback = TimeHistory(timelimit)
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0., patience=10)
    model.compile(optimizer='adam', metrics=['acc'],
                  loss='binary_crossentropy')
    history.append(model.fit_generator(generator(batchsize), steps_per_epoch=889//batchsize,
                   epochs=nepochs, verbose=2, validation_data=([xMulVal,xPanVal],yVal),callbacks=[time_callback, es]),
                  max_queue_size=2) # I get out of memory errors, so I reduce the queue
    # but the out of memory is probably due to the GPU rather than the CPU, so  it might not make any difference
    xaxis.append(time_callback.times)
    modelList.append(model) 

(99, 8, 325, 325) (99, 1, 1300, 1300) (99, 325, 325)


I need to downsample 1300 image so that I can combine it with the other one

This is a copy from tiramisu.py right now

In [3]:
# shared CNN parameters
REGULARIZER_L = 1e-4
DROPOUT_RATE = 0.2
PADDING ='same'
INITIALIZER = 'he_uniform'
POOLING = (2, 2)


def tiramisu(_input, blocks=[4, 5, 7, 10, 12], bottleneck=15,  # architecture of the tiramisu
    n_classes=1, input_shape=(8, 325, 325)):  # properties of the data

    ##########################
    # image input # I use my own input
    #_input = layers.Input(shape=input_shape)
    ##########################
    # conv layer
    x = layers.Convolution2D(48, (3, 3), strides=(1, 1),
                             padding=PADDING, kernel_initializer=INITIALIZER,
                             kernel_regularizer=regularizers.l2(REGULARIZER_L))(_input)
    ##########################
    # down path
    skips = []
    for nb in blocks:
        x = _dense_block(nb, x, end2end=True)
        skips.append(x)
        x = _transition_down(x)
    ##########################
    # bottleneck
    x = _dense_block(bottleneck, x)
    ##########################
    # up path
    for nb in blocks[::-1]:
        tmp = skips.pop() # need to do try except for odd number, can only pop once
        try:
            x = layers.Concatenate(axis=1)([_transition_up(x), tmp]) # axis=1 because I use channel first
        except ValueError:
            x = layers.Concatenate(axis=1)([layers.ZeroPadding2D(((1, 0), (1, 0)))(_transition_up(x)), tmp])
        x = _dense_block(nb, x)
    ##########################
    # conv layer
    x = layers.Convolution2D(n_classes, (1, 1), strides=(1, 1),
                             padding=PADDING, kernel_initializer=INITIALIZER,
                             kernel_regularizer=regularizers.l2(REGULARIZER_L))(x)
    ##########################
    # segmented image output
    x = layers.Activation('softmax')(x)
    #_output = layers.Reshape((-1, n_classes))(x)
    ####################################################
    # put it together
    #model = Model(inputs=_input, outputs=_output)
    #return model
    return layers.Reshape((325,325))(x) # I want to combine it with my own layers
    # I use num_classes = 1 since for 2 classes (road or not road), we only need one output.
    # The shape would be (1, 325, 325) so I reshape it.


def _layer(x):
    filters = 16  # the growth rate (# of feature maps added per layer)
    kernel = (3, 3)
    stride = (1, 1)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization(axis=1, center=False, scale=False)(x)
    x = layers.Convolution2D(filters, kernel, strides=stride,
                             padding=PADDING, kernel_initializer=INITIALIZER,
                             kernel_regularizer=regularizers.l2(REGULARIZER_L))(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    return x


def _dense_block(n_layers, x, end2end=False):
    # if end2end, will provide a path from input to output, as well as internal paths
    _these_layer_outputs = []
    _start = x
    # n-1 layers with their outputs concatted to their inputs
    for i in range(n_layers-1):
        lyr = _layer(x)
        _these_layer_outputs.append(lyr)
        x = layers.Concatenate(axis=1)([x, lyr]) # axis=0 because I use channel first
    # one more layer, then concatenate all of the layer outputs
    _these_layer_outputs.append(_layer(x))
    if end2end:
        _these_layer_outputs.append(_start)
    x = layers.Concatenate(axis=1)(_these_layer_outputs) # axis=0 because I use channel first
    return x


def _transition_down(x):
    filters = x.get_shape().as_list()[-1]  # same number of output feature maps as input
    kernel = (1, 1)
    stride = (1, 1)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization(axis=1, center=False, scale=False)(x)
    x = layers.Convolution2D(filters, kernel, strides=stride,
                             padding=PADDING, kernel_initializer=INITIALIZER,
                             kernel_regularizer=regularizers.l2(REGULARIZER_L))(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    x = layers.MaxPooling2D(pool_size=POOLING, strides=POOLING)(x)
    return x


def _transition_up(x):
    filters = x.get_shape().as_list()[-1]  # same number of output feature maps as input
    kernel = (3, 3)
    x = layers.Conv2DTranspose(filters, kernel, strides=POOLING,
                               padding=PADDING, kernel_initializer=INITIALIZER,
                               kernel_regularizer=regularizers.l2(REGULARIZER_L))(x)
    return x

In [4]:
K.set_image_data_format("channels_first")
def NN(XMul,XPan): # XMul is of shape (100, 8, 325, 325), XPan is (100, 1, 1300, 1300), output is (100, 1300, 1300)
    num_channel_Pan = 4
    
    x = layers.Conv2D(num_channel_Pan//2,(3, 3), strides=(1,1), padding=PADDING, kernel_initializer=INITIALIZER,
                               kernel_regularizer=regularizers.l2(REGULARIZER_L))(XPan) # need to do 2x2 pooling twice
    x = layers.Dropout(DROPOUT_RATE)(x)
    x = layers.MaxPooling2D(pool_size=POOLING, strides=POOLING)(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization(axis=1, center=False, scale=False)(x)
    x = layers.Conv2D(num_channel_Pan,(3, 3), strides=(1,1), padding=PADDING, kernel_initializer=INITIALIZER,
                               kernel_regularizer=regularizers.l2(REGULARIZER_L))(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    x = layers.MaxPooling2D(pool_size=POOLING, strides=POOLING)(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization(axis=1, center=False, scale=False)(x)
    
    x = layers.Concatenate(axis=1)([x,XMul]) # channel axis
    # I think it needs to be dividible 32 times, so it needs 320 or 352. I put a try except to do the padding just when needed
    x = tiramisu(x, blocks=[2, 3], bottleneck = 4, input_shape=(8+num_channel_Pan, 325, 325))
    # blocks was [4, 5, 7, 10, 12], but I have a ResourceExhaustedError and also I don't think we need the full tiramisu for
    # this simple task. bottleneck was 15
    return x

train(batchsize=1, nepochs=10, timelimit=np.inf)

Epoch 1/10
  file 0
     0 / 89
     1 / 89
     2 / 89
     3 / 89
     4 / 89
     5 / 89
     6 / 89
     7 / 89
     8 / 89
     9 / 89
     10 / 89


ResourceExhaustedError: OOM when allocating tensor with shape[1,258,325,325]
	 [[Node: gradients/conv2d_19/convolution_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@conv2d_19/convolution"], data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients/conv2d_19/convolution_grad/Shape, conv2d_19/kernel/read, gradients/conv2d_19/transpose_1_grad/transpose)]]

Caused by op 'gradients/conv2d_19/convolution_grad/Conv2DBackpropInput', defined at:
  File "C:\Users\nicol\Anaconda3\envs\tensorflowgpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\nicol\Anaconda3\envs\tensorflowgpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\nicol\Anaconda3\envs\tensorflowgpu\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\nicol\Anaconda3\envs\tensorflowgpu\lib\site-pa

I got ResourceExhaustedError  during 64 / 89 of file 1 for batchsize of 16. Memory is 94% (5.6 GB for python, 1 GB for google). I don't know why this file needs so much memory

I got ResourceExhaustedError  during 40 / 89 of file 0 for batchsize of 4. Memory is 94% (5.6 GB for python, 1 GB for google). I don't know why this file needs so much memory

# Model summary

bug: layers.BatchNormalization(axis=1, center=False, scale=False)(x) should use 0 parameters because of False

I should use image patch to reduce the memory need.

Or I can use a neuro network that is smaller than the Tiramisu

In [5]:
XMul = layers.Input(shape=xMulVal.shape[1:])
XPan = layers.Input(shape=xPanVal.shape[1:])

# Actual neural netowrk given by function NN
Y = NN(XMul,XPan)

model = Model(inputs=[XMul,XPan], outputs=Y)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 1, 1300, 1300) 0                                            
____________________________________________________________________________________________________
conv2d_21 (Conv2D)               (None, 2, 1300, 1300) 20          input_4[0][0]                    
____________________________________________________________________________________________________
dropout_19 (Dropout)             (None, 2, 1300, 1300) 0           conv2d_21[0][0]                  
____________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)   (None, 2, 650, 650)   0           dropout_19[0][0]                 
___________________________________________________________________________________________